In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import datetime as dt
import time

start = time.time()

# surgeries
surgId = []
description = []
duration = []
surgType = []

# surgeries needs
taskIds = []
surgNeedsId = []
skillNeeded = []
quantityNeeded = []
sortOrder = []
resDuration = []

# variables
roomCleaning = 15
roomPreparation = 15
kitCleaning = 240
bedStay = 720


treeSurg = ET.parse('surgeries_20190503.xml')
rootSurg = treeSurg.getroot()

taskId = 1
for elem in rootSurg.findall('order_line'):

    # Criacao da cirurgia com seus dados
    surgId = np.append(surgId, [elem.find('id').text])
    description = np.append(description, [elem.find('description').text])
    duration = np.append(duration, [int(elem.find('quantity').text)])
    surgType = np.append(surgType, [elem.find('orderlinekind').text])


    # Criacao das necessidades da cirurgia
    elemSurgId = elem.find('id').text
    surgDuration = int(elem.find('quantity').text)
    
    # ROOM
    if elem.find('required_room/roomid') != None:
        skillNeeded = np.append(skillNeeded, [elem.find('required_room/roomid').text])
    else:
        skillNeeded = np.append(skillNeeded, 'ROOM')
    surgNeedsId = np.append(surgNeedsId, elemSurgId)
    taskIds = np.append(taskIds, taskId)
    quantityNeeded = np.append(quantityNeeded, '1')
    sortOrder = np.append(sortOrder, '100')
    resDuration = np.append(resDuration, roomPreparation + surgDuration + roomCleaning)
    taskId += 1
    # KITS CME
    for elemKit in elem.findall('required_kits/required_kit'):
        taskIds = np.append(taskIds, taskId)
        surgNeedsId = np.append(surgNeedsId, elemSurgId)
        skillNeeded = np.append(skillNeeded, [elemKit.find('kitid').text])
        quantityNeeded = np.append(quantityNeeded, [elemKit.find('amount').text])
        sortOrder = np.append(sortOrder, [elemKit.find('sortorder').text])
        resDuration = np.append(resDuration, surgDuration + kitCleaning)
        taskId += 1

    # EQUIPAMENTOS
    for elemEquip in elem.findall('required_equipments/required_equipment'):
        taskIds = np.append(taskIds, taskId)
        surgNeedsId = np.append(surgNeedsId, elemSurgId)
        skillNeeded = np.append(skillNeeded, [elemEquip.find('equipmentid').text])
        quantityNeeded = np.append(quantityNeeded, [elemEquip.find('amount').text])
        sortOrder = np.append(sortOrder, [elemEquip.find('sortorder').text])
        resDuration = np.append(resDuration, surgDuration)
        taskId += 1
    
    # LEITO
    for elemLeito in elem.findall('required_bed'):
        taskIds = np.append(taskIds, taskId)
        surgNeedsId = np.append(surgNeedsId, elemSurgId)
        skillNeeded = np.append(skillNeeded, [elemLeito.find('bedid').text])
        quantityNeeded = np.append(quantityNeeded, [elemLeito.find('amount').text])
        sortOrder = np.append(sortOrder, [elemLeito.find('sortorder').text])
        resDuration = np.append(resDuration, bedStay)
        taskId += 1

    # EQUIPES
    for elemTeam in elem.findall('required_teams/required_team'):
        taskIds = np.append(taskIds, taskId)
        surgNeedsId = np.append(surgNeedsId, elemSurgId)
        skillNeeded = np.append(skillNeeded, [elemTeam.find('teamid').text])
        quantityNeeded = np.append(quantityNeeded, '1')
        timeDuration = elemTeam.find('duration').text
        minutesDuration = int(timeDuration[:2])*60 + int(timeDuration[3:5])
        resDuration = np.append(resDuration, minutesDuration)
        sortOrder = np.append(sortOrder, [elemTeam.find('sortorder').text])
        taskId += 1
    



data = {'id': surgId,
        'nome': description,
        'duration': duration,
        'type': surgType
        }

data1 = {'taskId': taskIds,
         'surgId': surgNeedsId,
         'need': skillNeeded,
         'quantity': quantityNeeded,
         'sortOrder': sortOrder,
         'duration': resDuration
        }
    
surg = pd.DataFrame(data)
surg['planned'] = False
surg.sort_values(by = ['duration'], ascending=False, inplace=True)
surg.reset_index(inplace=True)
surgeries = surg.copy()
surgNeeds = pd.DataFrame(data1)
surgNeeds = surgNeeds.sort_values(by=['surgId', 'sortOrder'])
surgNeeds['planned'] = False


####################################################################
# task relation
taskId_1 = []
taskId_2 = []
duration_wait = []
relationType = []

for i in pd.Series(surgNeeds.surgId).unique():
    for row in surgNeeds[surgNeeds.surgId == i].itertuples():
        if row.sortOrder[:1] == '1':
            taskIdRoom = row.taskId
        elif row.sortOrder[:1] == '2':
            taskId_1 = np.append(taskId_1, taskIdRoom)
            taskId_2 = np.append(taskId_2, row.taskId)
            duration_wait = np.append(duration_wait, roomPreparation)
            relationType = np.append(relationType, 'SS')
        elif row.sortOrder[:1] == '3':
            taskId_1 = np.append(taskId_1, taskIdRoom)
            taskId_2 = np.append(taskId_2, row.taskId)
            duration_wait = np.append(duration_wait, roomPreparation)
            relationType = np.append(relationType, 'SS')
        elif row.sortOrder[:1] == '4':
            if surg[surg.id == row.surgId].type.values[0] == 'SERIE' or surg[surg.id == row.surgId].type.values[0] == 'NORMAL':
                if row.sortOrder[1:3] == '00':
                    taskId_1 = np.append(taskId_1, taskIdRoom)
                    taskId_2 = np.append(taskId_2, row.taskId)
                    duration_wait = np.append(duration_wait, roomPreparation)
                    relationType = np.append(relationType, 'SS')

                    lastTaskId = row.taskId
                else:
                    taskId_1 = np.append(taskId_1, lastTaskId)
                    taskId_2 = np.append(taskId_2, row.taskId)
                    duration_wait = np.append(duration_wait, 0)
                    relationType = np.append(relationType, 'SE')

                    lastTaskId = row.taskId
            elif surg[surg.id == row.surgId].type.values[0] == 'PARALELO':
                taskId_1 = np.append(taskId_1, taskIdRoom)
                taskId_2 = np.append(taskId_2, row.taskId)
                duration_wait = np.append(duration_wait, roomPreparation)
                relationType = np.append(relationType, 'SS')

                allSurgs = surgNeeds[(surgNeeds.sortOrder.str.match('4')) & (surgNeeds.surgId == row.surgId)]
                biggerSurg = allSurgs[allSurgs.duration == allSurgs.duration.max()]
                biggerSurg.reset_index(inplace = True)
                lastTaskId = biggerSurg['taskId'][0]
        elif row.sortOrder[:1] == '5':
            taskId_1 = np.append(taskId_1, lastTaskId)
            taskId_2 = np.append(taskId_2, row.taskId)
            duration_wait = np.append(duration_wait, 0)
            relationType = np.append(relationType, 'SE')
        
taskRelDic =   {'taskId_01': taskId_1,
                'taskId_02': taskId_2,
                'durationWait': duration_wait,
                'relationType': relationType
                }
taskRelation = pd.DataFrame(taskRelDic)


####################################################################

# resources
resourceId = []
resDescription = []
resSkillGroup = []
resSkill = []

resourceIdAvail = []
resourceIdEntryAvail = []
resourceSkillAvail = []
resourceSkillGroupAvail = []
fromDate = []
tillDate = []


treeResource = ET.parse('resources_20190503.xml')
rootResource = treeResource.getroot()


for elem in rootResource.findall('resource'):
    elemResId = elem.find('id').text
    elemResDescrip = elem.find('description').text
    elemSkillConcat = ''
    for elemSkill in elem.findall('provide_properties/provide_property'):
        resourceId = np.append(resourceId, elemResId)
        resDescription = np.append(resDescription, elemResDescrip)
        resSkillGroup = np.append(resSkillGroup, [elemSkill.find('property_typeid').text])
        resSkill = np.append(resSkill, [elemSkill.find('value/id').text])
        elemSkillConcat = elemSkillConcat + elemSkill.find('value/id').text + ';'
        elemSkillGroup = elemSkill.find('property_typeid').text
    if elem.find('availabilities/availability/amount') != None:
        resAmount = int(elem.find('availabilities/availability/amount').text)
        while resAmount > 0:
            for elemAvail in elem.findall('availabilities/availability'):
                resourceIdAvail = np.append(resourceIdAvail, elemResId)
                resourceIdEntryAvail = np.append(resourceIdEntryAvail, elemResId + '_' + str(resAmount))
                resourceSkillAvail = np.append(resourceSkillAvail, elemSkillConcat)
                resourceSkillGroupAvail = np.append(resourceSkillGroupAvail, elemSkillGroup)
                if elemAvail.find('fromdate') != None:
                    fromDate = np.append(fromDate, pd.to_datetime(elemAvail.find('fromdate').text))
                else:
                    fromDate = np.append(fromDate, pd.to_datetime(dt.date.today()) + dt.timedelta(days=2))

                if elemAvail.find('tilldate') != None:
                    tillDate = np.append(tillDate, pd.to_datetime(elemAvail.find('tilldate').text))
                else:
                    tillDate = np.append(tillDate, pd.to_datetime(elemAvail.find('fromdate').text) + dt.timedelta(days=2))   
            resAmount -= 1

    

data2 = {'id': resourceId,
         'name': resDescription,
         'skillGroup': resSkillGroup,
         'skill': resSkill
         }

res = pd.DataFrame(data2)

data5 = {   'resourceId': resourceIdAvail,
            'resourceIdEntry': resourceIdEntryAvail,
            'fromDate': fromDate,
            'tillDate': tillDate,
            'skill': resourceSkillAvail,
            'skillGroup': resourceSkillGroupAvail
             }

resAvail = pd.DataFrame(data5)

resourcePlanning = pd.DataFrame(columns=['resourceIdEntry', 'fromDate', 'tillDate', 'surgId', 'taskId', 'resourceId'])

end = time.time()

print(end - start)
# seleção de todas as salas que possuem alguma disponibilidade
availaR = resAvail.loc[(resAvail['skillGroup'] == 'ROOMS') & (resAvail['fromDate'] < resAvail['tillDate'])] 
# seleção do recurso com disponibilidade primeiro:
availaR_min = availaR.sort_values(by=['fromDate'])
# pegar a primeira sala da lista anterior, em caso de ter mais de uma.
idx_ROOM = 0
i_ROOM = availaR_min.iloc[idx_ROOM]
t = i_ROOM['fromDate']


hasRoom = True


while (surg.shape[0] > 0) and (hasRoom == True):
    ## Copia do vetor de cirurgias para manipulacao no proximo while
   
    surg_t = surg.copy()
    nextTimeResAvail = pd.DataFrame(columns=['surgId', 'fromDate', 'duration'])
    surgPlanned_t = False
    

    while (surg_t.shape[0] > 0) and (surgPlanned_t == False):
        ## Selecionar primeira cirurgia da fila de prioridade:
        surg_it = surg_t.iloc[0]
        surg_it_id = surg_it.loc['id']
        ## Verificar se todos os recursos da cirurgia atual possuem disponibilidade
        surgNeeds_it = surgNeeds.loc[surgNeeds['surgId'] == surg_it_id]
        blockFound = False
        resAvail_it = resAvail.copy()
        resourcePlanning_it = resourcePlanning.copy()
        for row in surgNeeds_it.itertuples():
##            print(row.need)
##            print(surgNeeds_it)
            if blockFound == False:
##                print('not blocked')
                if row.need[:4] == 'ROOM':
                    if row.need in i_ROOM.skill:
                        if (t + dt.timedelta(minutes=row.duration)) >= i_ROOM.tillDate:
                            blockFound = True
                        else:
                            surgNeeds_it.loc[surgNeeds_it.taskId == row.taskId, 'planned'] = True
                            resAvail_it.loc[resAvail_it.resourceIdEntry == i_ROOM.resourceIdEntry, 'fromDate'] = (t + dt.timedelta(minutes=row.duration))     
                            resourcePlanning_it = resourcePlanning_it.append({'resourceIdEntry': i_ROOM.resourceIdEntry, 'fromDate': t, 'tillDate': (t + dt.timedelta(minutes=row.duration)), 'surgId': row.surgId, 'taskId': row.taskId, 'resourceId': i_ROOM.resourceId}, ignore_index=True)
                            resourcePlanning_it.sort_index(inplace=True)
                    else:
                        blockFound = True
##                        print('blocked on room')
                        
                else:
                    taskRelation_it = taskRelation[row.taskId == taskRelation.taskId_02]
                    if taskRelation_it.relationType.values[0] == 'SS':
                        startTask = pd.to_datetime(resourcePlanning_it.loc[resourcePlanning_it.taskId == taskRelation_it.taskId_01.values[0], 'fromDate'].values[0]) + dt.timedelta(minutes=taskRelation_it.durationWait.values[0])
                        finishTask = startTask + dt.timedelta(minutes=row.duration)
                    elif taskRelation_it.relationType.values[0] == 'SE':
                        startTask = pd.to_datetime(resourcePlanning_it.loc[resourcePlanning_it.taskId == taskRelation_it.taskId_01.values[0], 'tillDate'].values[0]) + dt.timedelta(minutes=taskRelation_it.durationWait.values[0])
                        finishTask = startTask + dt.timedelta(minutes=row.duration)

                    resFound = resAvail_it[(resAvail_it.skill.str.contains(row.need)) & (startTask >= resAvail_it.fromDate) & (finishTask <= resAvail_it.tillDate)]
                    if resFound.shape[0] >= 1:
                        resource = resFound.iloc[0]
                        surgNeeds_it.loc[surgNeeds_it.taskId == row.taskId, 'planned'] = True  
                        resourcePlanning_it = resourcePlanning_it.append({'resourceIdEntry': resource.resourceIdEntry, 'fromDate': startTask, 'tillDate': finishTask, 'surgId': row.surgId, 'taskId': row.taskId, 'resourceId': resource.resourceId}, ignore_index=True)
                    
                        if startTask == resource.fromDate:
                            resAvail_it.loc[(resAvail_it.resourceIdEntry == resource.resourceIdEntry) & (resAvail_it.fromDate == resource.fromDate), 'fromDate'] = finishTask
                        else:
                            resAvail_it.loc[(resAvail_it.resourceIdEntry == resource.resourceIdEntry) & (resAvail_it.tillDate == resource.tillDate), 'tillDate'] = startTask
                            resAvail_it = resAvail_it.append({'resourceId': resource.resourceId, 'resourceIdEntry': resource.resourceIdEntry, 'fromDate': finishTask, 'tillDate': resource.tillDate, 'skill': resource.skill, 'skillGroup': resource.skillGroup}, ignore_index=True)
                        
                    else:
                        blockFound = True
##                        print('blocked on resource')
                        resNextDate = resAvail_it[(resAvail_it.skill.str.contains(row.need)) & (startTask < resAvail_it.fromDate)]
                        if resNextDate.shape[0] >=1:
                            firstResAvail = resNextDate[resNextDate.fromDate == resNextDate.fromDate.min()].iloc[0]
                            nextTime = pd.to_datetime(firstResAvail.fromDate) - dt.timedelta(minutes=taskRelation_it.durationWait.values[0])
                            nextTimeResAvail = nextTimeResAvail.append({'fromDate': nextTime, 'surgId': surg_it_id, 'duration': surg_it.duration}, ignore_index=True)
                        else:
                            surg = surg[surg.id != surg_it_id]
                        

        if blockFound == False:
##            print(surg)
            resAvail = resAvail_it
            resourcePlanning = resourcePlanning_it
            surgeries.loc[surgeries.id == surg_it_id, 'planned'] = True
            surg = surg[surg.id != surg_it_id]
            surgPlanned_t = True
            # seleção de todas as salas que possuem alguma disponibilidade
            availaR = resAvail.loc[(resAvail['skillGroup'] == 'ROOMS') & (resAvail['fromDate'] < resAvail['tillDate']) & (resAvail['fromDate'] >= t)] 
            # seleção do recurso com disponibilidade primeiro:
            availaR_min = availaR.sort_values(by=['fromDate'])
            # pegar a primeira sala da lista anterior, em caso de ter mais de uma.
            idx_ROOM = 0
            i_ROOM = availaR_min.iloc[idx_ROOM]
            t = i_ROOM['fromDate']
 
        else:
            surg_t = surg_t[surg_t.id != surg_it_id]
            if surg_t.shape[0] == 0 and surg.shape[0] != 0:
                if nextTimeResAvail.shape[0] != 0:
                    nextTimeResAvail = nextTimeResAvail.sort_values(by = ['fromDate', 'duration'], ascending=[True, False])
                    t = nextTimeResAvail.iloc[0,1]
                    #t += dt.timedelta(minutes=1)
                else:
                    resAvail.loc[resAvail.resourceIdEntry == i_ROOM.resourceIdEntry, 'fromDate'] = resAvail.tillDate
                    # seleção de todas as salas que possuem alguma disponibilidade
                    availaR = resAvail.loc[(resAvail['skillGroup'] == 'ROOMS') & (resAvail['fromDate'] < resAvail['tillDate'])]
                    if availaR.shape[0] == 0:
                        hasRoom = False
                    else:
                        # seleção do recurso com disponibilidade primeiro:
                        availaR_min = availaR.sort_values(by=['fromDate'])
                        # pegar a primeira sala da lista anterior, em caso de ter mais de uma.
                        idx_ROOM = 0
                        i_ROOM = availaR_min.iloc[idx_ROOM]
                        t = i_ROOM['fromDate']

##import plotly
##plotly.tools.set_credentials_file(username='dougmart', api_key='z3YRoxBeo8Lj1i1s8Ylo')
##import plotly.plotly as py
##import plotly.figure_factory as ff
##
##
##
##
##
##
##df = []
##for row in resourcePlanning.itertuples():
##    if row.resourceId[:4] == 'ROOM':
##        df.append(dict(Task = row.resourceIdEntry, Start=row.fromDate, Finish=row.tillDate))
##
##
##
##fig = ff.create_gantt(df, group_tasks=True)
##py.iplot(fig, filename='gantt-hours', world_readable=True)


# create the file structure

data = ET.Element('planwise')
data.set('updatemode', 'full')
data.set('timestamp', dt.datetime.now().strftime('%Y-%m-%dT%X'))
items = ET.SubElement(data, 'cirurgias')


for rowSurg in surgeries[surgeries.planned == True].itertuples():
    item = ET.SubElement(items, 'cirurgia')
    itemSurgId = ET.SubElement(item, 'id')
    itemSurgId.text = rowSurg.id
    itemRoomId = ET.SubElement(item, 'sala_id')
    itemRoomId.text = resourcePlanning.loc[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('ROOM')), 'resourceId'].values[0]
    itemStart = ET.SubElement(item, 'inicio')
    startSurg = resourcePlanning.loc[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('ROOM')), 'fromDate'].values[0]
    itemStart.text = pd.to_datetime(startSurg).strftime('%Y-%m-%dT%X')
    itemFinish = ET.SubElement(item, 'fim')
    finishSurg = resourcePlanning.loc[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('ROOM')), 'tillDate'].values[0]
    itemFinish.text = pd.to_datetime(finishSurg).strftime('%Y-%m-%dT%X')
    itemBedId = ET.SubElement(item, 'leito_id')
    itemBedId.text = resourcePlanning.loc[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('BED')), 'resourceId'].values[0]
    
    nofTeams = resourcePlanning[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('TEAM'))].shape[0]
    teamStr = ''
    for rowTeam in resourcePlanning[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('TEAM'))].itertuples():
        teamStr = teamStr + rowTeam.resourceId
        nofTeams -= 1
        if nofTeams > 0:
            teamStr = teamStr + ','
    itemTeamId = ET.SubElement(item, 'team_id')
    itemTeamId.text = teamStr

    nofKits = resourcePlanning[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('KIT'))].shape[0]
    kitsStr = ''
    for rowKit in resourcePlanning[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('KIT'))].itertuples():
        kitsStr = kitsStr + rowKit.resourceId
        nofKits -= 1
        if nofKits > 0:
            kitsStr = kitsStr + ','
    itemKitId = ET.SubElement(item, 'kit_id')
    itemKitId.text = kitsStr

    nofEquips = resourcePlanning[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('EQUIPMENT'))].shape[0]
    equipsStr = ''
    for rowEquip in resourcePlanning[(resourcePlanning.surgId == rowSurg.id) & (resourcePlanning.resourceId.str.match('EQUIPMENT'))].itertuples():
        equipsStr = equipsStr + rowEquip.resourceId
        nofEquips -= 1
        if nofEquips > 0:
            equipsStr = equipsStr + ','
    itemEquipId = ET.SubElement(item, 'equipamento_id')
    itemEquipId.text = equipsStr

# create a new XML file with the results
mydata = ET.tostring(data)  
myfile = open("items2.xml", "wb")  
myfile.write(mydata)  
myfile.close()





##resAvail.loc[resAvail.skillGroup == 'ROOMS', ['fromDate', 'tillDate']]
##resAvail.loc[resAvail.resourceId == 'TEAM_66', ['fromDate', 'tillDate']]


0.8477683067321777


C:\Users\douglasm\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [2]:

import plotly
import plotly.figure_factory as ff

In [5]:
df = []
for row in resourcePlanning.itertuples():
    if row.resourceId[:4] == 'ROOM':
        df.append(dict(Task = row.resourceIdEntry, Start=row.fromDate, Finish=row.tillDate))

        
fig = ff.create_gantt(df, group_tasks=True)
fig.show()

In [7]:
df

[{'Task': 'ROOM_86_1',
  'Start': Timestamp('2019-05-07 00:00:00'),
  'Finish': Timestamp('2019-05-07 02:12:00')},
 {'Task': 'ROOM_87_1',
  'Start': Timestamp('2019-05-07 00:00:00'),
  'Finish': Timestamp('2019-05-07 02:00:00')},
 {'Task': 'ROOM_89_1',
  'Start': Timestamp('2019-05-07 00:00:00'),
  'Finish': Timestamp('2019-05-07 01:57:00')},
 {'Task': 'ROOM_90_1',
  'Start': Timestamp('2019-05-07 00:00:00'),
  'Finish': Timestamp('2019-05-07 01:50:00')},
 {'Task': 'ROOM_90_1',
  'Start': Timestamp('2019-05-07 01:50:00'),
  'Finish': Timestamp('2019-05-07 03:40:00')},
 {'Task': 'ROOM_89_1',
  'Start': Timestamp('2019-05-07 01:57:00'),
  'Finish': Timestamp('2019-05-07 03:47:00')},
 {'Task': 'ROOM_87_1',
  'Start': Timestamp('2019-05-07 02:00:00'),
  'Finish': Timestamp('2019-05-07 03:38:00')},
 {'Task': 'ROOM_86_1',
  'Start': Timestamp('2019-05-07 02:12:00'),
  'Finish': Timestamp('2019-05-07 03:50:00')},
 {'Task': 'ROOM_87_1',
  'Start': Timestamp('2019-05-07 03:38:00'),
  'Finish': 